# ChatPromptTemplate trong LangChain

## Giới thiệu

Trong notebook này, chúng ta sẽ tìm hiểu về:
- **ChatPromptTemplate** là gì và sự khác biệt với PromptTemplate
- Cách xây dựng các prompt có vai trò (System, Human, AI)
- Tích hợp với ChatAnthropic để tạo các cuộc hội thoại
- Các kỹ thuật nâng cao với ChatPromptTemplate

## 1. Sự khác biệt giữa PromptTemplate và ChatPromptTemplate

### PromptTemplate
- Được thiết kế cho **completion models** (text-in, text-out)
- Chỉ có một chuỗi văn bản đầu vào
- Phù hợp cho các tác vụ đơn giản

### ChatPromptTemplate
- Được thiết kế cho **chat models** (messages-in, message-out)
- Hỗ trợ nhiều vai trò: System, Human, AI
- Phù hợp cho các cuộc hội thoại phức tạp

In [ ]:
# Import các thư viện cần thiết
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
import os
from dotenv import load_dotenv

# Load biến môi trường
load_dotenv()

# Khởi tạo ChatAnthropic
chat = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.7,
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

## 2. Tạo ChatPromptTemplate cơ bản

In [ ]:
# Cách 1: Sử dụng from_messages
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý AI chuyên về {topic}."),
    ("human", "Hãy giải thích về {concept} một cách đơn giản.")
])

# Format prompt với các biến
messages = chat_template.format_messages(
    topic="lập trình Python",
    concept="list comprehension"
)

print("Messages được tạo:")
for msg in messages:
    print(f"{msg.__class__.__name__}: {msg.content}")

In [ ]:
# Gửi messages đến ChatAnthropic
response = chat.invoke(messages)
print("\nPhản hồi từ Claude:")
print(response.content)

## 3. Các vai trò trong ChatPromptTemplate

### System Message
- Định nghĩa vai trò và hành vi của AI
- Thiết lập context và constraints

### Human Message
- Đại diện cho input từ người dùng
- Chứa câu hỏi hoặc yêu cầu

### AI Message
- Đại diện cho phản hồi trước đó của AI
- Hữu ích cho few-shot learning

In [ ]:
# Template với nhiều vai trò
multi_role_template = ChatPromptTemplate.from_messages([
    ("system", """Bạn là một giáo viên dạy tiếng Việt cho người nước ngoài.
    Hãy giải thích ngữ pháp một cách rõ ràng và cho ví dụ cụ thể."""),
    ("human", "Từ '{word}' có nghĩa là gì?"),
    ("ai", "Tôi sẽ giải thích từ '{word}' cho bạn."),
    ("human", "Cho tôi ví dụ sử dụng từ này trong câu.")
])

# Format với từ cụ thể
messages = multi_role_template.format_messages(word="xin chào")

# In ra để xem cấu trúc
for i, msg in enumerate(messages):
    print(f"Message {i+1} ({msg.__class__.__name__}):")
    print(f"  {msg.content}\n")

## 4. Few-shot Learning với ChatPromptTemplate

In [ ]:
# Few-shot template cho phân loại cảm xúc
few_shot_template = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một chuyên gia phân tích cảm xúc. Phân loại câu thành: Tích cực, Tiêu cực, hoặc Trung lập."),
    # Ví dụ 1
    ("human", "Hôm nay trời đẹp quá!"),
    ("ai", "Cảm xúc: Tích cực"),
    # Ví dụ 2
    ("human", "Tôi rất buồn vì mất việc."),
    ("ai", "Cảm xúc: Tiêu cực"),
    # Ví dụ 3
    ("human", "Hôm nay là thứ ba."),
    ("ai", "Cảm xúc: Trung lập"),
    # Câu cần phân loại
    ("human", "{text}")
])

# Test với các câu khác nhau
test_sentences = [
    "Tôi rất vui khi được gặp bạn!",
    "Dịch vụ này thật tệ.",
    "Giá của sản phẩm là 100.000đ."
]

for sentence in test_sentences:
    messages = few_shot_template.format_messages(text=sentence)
    response = chat.invoke(messages)
    print(f"Câu: '{sentence}'")
    print(f"Phân loại: {response.content}\n")

## 5. Sử dụng MessagesPlaceholder cho lịch sử hội thoại

In [ ]:
# Template với placeholder cho lịch sử
conversation_template = ChatPromptTemplate.from_messages([
    ("system", """Bạn là một trợ lý AI thân thiện. 
    Hãy nhớ context từ các tin nhắn trước đó và trả lời một cách nhất quán."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# Tạo lịch sử hội thoại
chat_history = [
    HumanMessage(content="Tên tôi là Nam."),
    AIMessage(content="Xin chào Nam! Rất vui được gặp bạn."),
    HumanMessage(content="Tôi đang học lập trình Python."),
    AIMessage(content="Tuyệt vời! Python là ngôn ngữ rất phù hợp cho người mới bắt đầu.")
]

# Hỏi câu hỏi mới với context
messages = conversation_template.format_messages(
    chat_history=chat_history,
    question="Bạn có nhớ tên tôi không?"
)

response = chat.invoke(messages)
print("Câu hỏi: Bạn có nhớ tên tôi không?")
print(f"Trả lời: {response.content}")

## 6. Template động với partial variables

In [ ]:
from datetime import datetime

# Template với biến động
dynamic_template = ChatPromptTemplate.from_messages([
    ("system", """Bạn là một trợ lý AI. 
    Hôm nay là {current_date}.
    Hãy trả lời dựa trên thông tin thời gian hiện tại."""),
    ("human", "{question}")
])

# Partial template với ngày hiện tại
partial_template = dynamic_template.partial(
    current_date=datetime.now().strftime("%d/%m/%Y")
)

# Sử dụng template đã partial
questions = [
    "Hôm nay là ngày gì?",
    "Còn bao nhiêu ngày nữa đến Tết Nguyên Đán 2025?"
]

for q in questions:
    messages = partial_template.format_messages(question=q)
    response = chat.invoke(messages)
    print(f"Q: {q}")
    print(f"A: {response.content}\n")

## 7. Template phức tạp cho use case thực tế

In [ ]:
# Template cho chatbot hỗ trợ khách hàng
customer_support_template = ChatPromptTemplate.from_messages([
    ("system", """Bạn là nhân viên hỗ trợ khách hàng của công ty {company_name}.
    
    Thông tin công ty:
    - Lĩnh vực: {industry}
    - Chính sách hoàn trả: {return_policy}
    
    Hướng dẫn:
    1. Luôn lịch sự và chuyên nghiệp
    2. Giải quyết vấn đề của khách hàng
    3. Đề xuất giải pháp phù hợp
    4. Nếu không thể giải quyết, hướng dẫn liên hệ bộ phận phù hợp
    """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{customer_message}")
])

# Cấu hình cho một công ty cụ thể
configured_template = customer_support_template.partial(
    company_name="TechViet Solutions",
    industry="Phần mềm và dịch vụ IT",
    return_policy="Hoàn tiền 100% trong 30 ngày nếu không hài lòng"
)

# Mô phỏng cuộc hội thoại
customer_queries = [
    "Tôi muốn biết về chính sách hoàn trả của công ty.",
    "Phần mềm của các bạn có hỗ trợ tiếng Việt không?",
    "Tôi gặp lỗi khi cài đặt, làm sao để khắc phục?"
]

conversation_history = []

for query in customer_queries:
    print(f"Khách hàng: {query}")
    
    messages = configured_template.format_messages(
        chat_history=conversation_history,
        customer_message=query
    )
    
    response = chat.invoke(messages)
    print(f"Hỗ trợ: {response.content}\n")
    
    # Cập nhật lịch sử
    conversation_history.append(HumanMessage(content=query))
    conversation_history.append(AIMessage(content=response.content))

## 8. So sánh với PromptTemplate truyền thống

In [ ]:
from langchain_core.prompts import PromptTemplate

# PromptTemplate truyền thống
simple_prompt = PromptTemplate(
    template="Dịch câu sau sang tiếng Anh: {vietnamese_text}",
    input_variables=["vietnamese_text"]
)

# ChatPromptTemplate tương đương
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một dịch giả chuyên nghiệp Việt-Anh."),
    ("human", "Dịch câu sau sang tiếng Anh: {vietnamese_text}")
])

# So sánh output
test_text = "Xin chào, rất vui được gặp bạn"

print("=== PromptTemplate Output ===")
print(simple_prompt.format(vietnamese_text=test_text))

print("\n=== ChatPromptTemplate Output ===")
messages = chat_prompt.format_messages(vietnamese_text=test_text)
for msg in messages:
    print(f"{msg.__class__.__name__}: {msg.content}")

# Gọi với ChatAnthropic
response = chat.invoke(messages)
print(f"\nKết quả dịch: {response.content}")

## 9. Best Practices và Tips

### 1. Thiết kế System Message hiệu quả
- Rõ ràng về vai trò và nhiệm vụ
- Cung cấp constraints và guidelines
- Không quá dài, tập trung vào điều quan trọng

### 2. Sử dụng Few-shot khi cần
- Cung cấp 2-3 ví dụ chất lượng cao
- Đảm bảo ví dụ đa dạng và representative

### 3. Quản lý Chat History
- Giới hạn độ dài history để tránh tốn token
- Lưu trữ và tóm tắt các cuộc hội thoại dài

In [ ]:
# Ví dụ về template với best practices
optimized_template = ChatPromptTemplate.from_messages([
    ("system", """Vai trò: Chuyên gia tư vấn {domain}
    
    Nguyên tắc:
    - Trả lời chính xác, dựa trên kiến thức chuyên môn
    - Nếu không chắc chắn, hãy nói rõ
    - Giải thích phức tạp theo cách dễ hiểu
    """),
    MessagesPlaceholder(variable_name="examples", optional=True),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("human", "{question}")
])

# Sử dụng với các biến khác nhau
examples = [
    HumanMessage(content="Machine Learning là gì?"),
    AIMessage(content="Machine Learning là một nhánh của AI cho phép máy tính học từ dữ liệu mà không cần lập trình cụ thể.")
]

messages = optimized_template.format_messages(
    domain="Trí tuệ nhân tạo",
    examples=examples,
    history=[],
    question="Deep Learning khác gì với Machine Learning?"
)

response = chat.invoke(messages)
print(response.content)

## Tổng kết

Trong notebook này, chúng ta đã học:

1. **Sự khác biệt cơ bản** giữa PromptTemplate và ChatPromptTemplate
2. **Các vai trò** trong chat: System, Human, AI
3. **Few-shot learning** với ChatPromptTemplate
4. **MessagesPlaceholder** để quản lý lịch sử hội thoại
5. **Partial variables** cho template động
6. **Use cases thực tế** như customer support chatbot

ChatPromptTemplate là công cụ mạnh mẽ cho việc xây dựng các ứng dụng chat phức tạp với LangChain và Claude!